In [2]:

import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import pandas as pd
import math
from numpy import diff
from numpy import gradient
import numpy as np
from scipy import pi
from scipy import fft
from scipy.fft import fft, fftfreq
from scipy import signal
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots


In [7]:
from gps import get_data

In [14]:
df1=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02.miq', 
                skiprows=57, skipfooter=35,names=['sample', 'Lat','Lon', 'Alt','Time', 'Speed'])

#postion,sensors  = get_data(mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02)
df=pd.read_csv('mtb_phd/drive-download-20230823T084905Z-001/BrakeAce 20230315180854 Minerals +Torque.csv') #,usecols=[1,2,3,4,5], 
               #names=( 'fork','rearWheel','frontForce','shock','rearForce'), header=0)
               
df

/tmp/ipykernel_22172/1410175824.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df1=pd.read_csv('mtb_phd/minerals/OPPOFindX5Pro-03-15-02/OPPOFindX5Pro-03-15-02.miq',


,Time (s),Distance (m),Latitude (deg),Longitude (deg),Elevation (m),Speed (km/h),Front Brake Power (W),Rear Brake Power (W),Front Brake Torque (Nm),Rear Brake Torque (Nm)
0,38.2,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
1,38.3,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
2,38.4,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
3,38.5,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
4,38.6,0,-38.17087,176.30223,636,0.1,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2302,268.4,1106,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2303,268.5,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2304,268.6,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0
2305,268.7,1107,-38.16651,176.30820,460,3.9,0.0,0.0,0.0,0.0


In [27]:
df.columns

Index(['Time (s)', ' Distance (m)', ' Latitude (deg)', ' Longitude (deg)',
       ' Elevation (m)', ' Speed (km/h)', ' Front Brake Power (W)',
       ' Rear Brake Power (W)', ' Front Brake Torque (Nm)',
       ' Rear Brake Torque (Nm)'],
      dtype='object')

In [16]:
    # size=len(df1.index)
    # timedata,interval=np.linspace(0,size/200,num=size,retstep=True, endpoint=False) #create timestamps
    # df1['time']=np.round(timedata,3) # add to dataframe, round to 0.001s
df1['elapsed_time'] = df1['Time'] - df1['Time'][0]

In [17]:
df1

,sample,Lat,Lon,Alt,Time,Speed,elapsed_time
0,3,-38.170776,176.302207,634.598145,1.678857e+09,2.345496,0.000
1,4,-38.170746,176.302225,634.472534,1.678857e+09,2.939015,1.195
2,5,-38.170705,176.302245,634.472534,1.678857e+09,2.838630,1.792
3,6,-38.170700,176.302249,633.794678,1.678857e+09,2.679965,2.298
4,7,-38.170688,176.302255,627.541016,1.678857e+09,2.634642,2.867
...,...,...,...,...,...,...,...
293,296,-38.166518,176.308504,454.841675,1.678857e+09,0.193711,233.645
294,297,-38.166518,176.308506,455.514648,1.678857e+09,0.135244,234.242
295,298,-38.166518,176.308508,455.976929,1.678857e+09,0.086141,234.839
296,299,-38.166517,176.308509,455.976929,1.678857e+09,0.062891,235.342


In [30]:
a = np.array(df1['Speed'])
b = np.array(df[' Speed (km/h)'])
print(a)
print(b)

[2.345496 2.939015 2.83863  2.679965 2.634642 2.45583  2.316443 2.331259
 2.488754 2.894351 3.911563 4.3605   4.653504 4.156574 3.677057 4.083106
 4.700343 4.588739 4.655656 4.591189 4.59477  4.507096 4.492047 4.473976
 4.535911 4.726426 4.716155 5.027205 5.068179 5.03299  5.039606 4.882174
 4.84493  4.809098 4.839824 4.871627 4.862751 4.796155 5.073198 5.182613
 5.152217 5.086677 5.320558 5.507696 5.635052 5.785287 5.929749 6.013143
 6.75605  7.435185 6.704595 7.247977 7.008165 6.564957 6.434369 5.549766
 5.139004 5.804101 6.21325  5.915659 5.930802 5.445815 4.727246 5.405766
 4.428875 3.413228 3.855735 4.41871  4.494761 4.247653 3.725893 3.109849
 3.344316 3.453915 3.380095 3.385788 3.895834 3.95691  4.35822  6.380273
 4.612965 5.847798 5.757275 6.840067 7.741833 6.261108 5.843708 6.700315
 8.047392 8.866114 6.053722 8.697694 6.751791 4.230185 5.993243 5.760289
 4.936899 6.598648 9.155602 8.670185 7.834826 3.91552  2.949532 4.51855
 5.135147 5.223012 6.189062 7.247907 7.424869 7.7621

In [29]:
distance, path = fastdtw(a, b, dist=euclidean)
 print(distance)

ValueError: Input vector should be 1-D.